In [1]:
from el_toolkit.document import Document
from el_toolkit.lkb.lexical_knowledge_base import Lexical_Knowledge_Base
from transformers import BertTokenizer


lkb = Lexical_Knowledge_Base.read_json("data/knowledge_datasets/mesh_subset.json")

biomed_tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path='monologg/biobert_v1.1_pubmed',
                                                 do_lower_case=False, cache_dir=None)
doc = Document.read_json("/Users/coltonflowers/Work/TREND_repos/BioMedical-EL/tests/test_data/test_docs/without_overlaps/17_mentions.json")
doc.jupyter_display(lkb)
for segmented_doc in doc.segment(biomed_tokenizer,8):
  segmented_doc.jupyter_display(lkb)
  print("-----------------------------------")

Creating conceptual edge index
Creating lexical edge index


Mentions Omitted: 0


-----------------------------------


-----------------------------------


-----------------------------------


In [2]:
from el_toolkit.entity_linkers.dual_embedder.featurizer import BertDualEmbedderTrainFeaturizer
from el_toolkit.entity_linkers.dual_embedder.concept_embedder import BertConceptEmbedder
from el_toolkit.entity_linkers.dual_embedder.document_embedder import DocumentEmbedder
from el_toolkit.entity_linkers.dual_embedder.entity_linker import DualEmbedderEntityLinker
from el_toolkit.entity_linkers.dual_embedder.model import BertMentionDetectorModel
from el_toolkit.entity_linkers.dual_embedder.model import BertDualEmbedderModel
from transformers import BertModel
import copy

bert_mention = BertModel.from_pretrained("monologg/biobert_v1.1_pubmed")
bert_candidate = copy.deepcopy(bert_mention)
concept_embedder = BertConceptEmbedder(lkb,bert_model=bert_mention,tokenizer=biomed_tokenizer,max_seq_len=256)
span_detector = BertMentionDetectorModel(bert_mention)
document_embedder = DocumentEmbedder(span_detector=span_detector,tokenizer=biomed_tokenizer,max_seq_len=256)
dual_embedder_model = BertDualEmbedderModel(bert_candidate,bert_mention)
dual_embedder = DualEmbedderEntityLinker(concept_embedder,document_embedder,dual_embedder_model)
featurizer = BertDualEmbedderTrainFeaturizer(lkb,dual_embedder)
docs = doc.segment(biomed_tokenizer,8) 

Some weights of the model checkpoint at monologg/biobert_v1.1_pubmed were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Mentions Omitted: 0


In [3]:
dual_embedder.train(docs,num_random_negatives=4)

Epoch:   0%|          | 0/100 [00:00<?, ?it/s]/Users/coltonflowers/opt/anaconda3/envs/notebook/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch:   3%|▎         | 3/100 [03:01<1:36:32, 59.72s/it]

In [ ]:
dataset[0][3]

In [ ]:
dataset[0][2]

In [ ]:
doc_token_ids,doc_token_mask,mention_start_indices,mention_end_indices,_,_  = dual_embedder.document_embedder.encode_document(doc)

In [ ]:
lkb = Lexical_Knowledge_Base.read_json("data/knowledge_datasets/trend.json")

In [ ]:
featurizer.featurize